In [1]:
import torch

In [2]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [3]:
a[1]

tensor(1.)

In [4]:
a[2] = 2.
a

tensor([1., 1., 2.])

In [5]:
points = torch.tensor([4., 1., 5., 3., 2., 1.])
points

tensor([4., 1., 5., 3., 2., 1.])

In [6]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [7]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [8]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [9]:
points.shape

torch.Size([3, 2])

In [10]:
points.size()

torch.Size([3, 2])

In [11]:
points[2, 1]

tensor(1.)

In [12]:
points[1]

tensor([5., 3.])

In [13]:
points[:, 1]

tensor([1., 3., 1.])

In [14]:
points[None] # note added dim

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

### Named tensors

In [15]:
img_t = torch.randn(3, 5, 5) # [channels, rows, cols]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t = torch.randn(2, 3, 5, 5) # [batch, chan, rows, cols]

In [16]:
img_grey_naive = img_t.mean(-3)
batch_grey_naive = batch_t.mean(-3)
img_grey_naive.shape, batch_grey_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [17]:
weights.unsqueeze(-1)

tensor([[0.2126],
        [0.7152],
        [0.0722]])

In [18]:
weights.unsqueeze_(-1)

tensor([[0.2126],
        [0.7152],
        [0.0722]])

In [19]:
weights.unsqueeze(-1).unsqueeze_(-1)

tensor([[[[0.2126]]],


        [[[0.7152]]],


        [[[0.0722]]]])

In [20]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weight = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights
img_grey_weighted = img_weights.sum(-3)
batch_grey_weights = batch_weights.sum(-3)
[x.shape for x in (batch_weights, batch_t, unsqueezed_weights)]

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 0

In [ ]:
weights_named = torch.tensor([0.2126, 0.7163, 0.0733], 
                             names=['channels'])
weights_named

In [ ]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)
print('batch named:', batch_named.shape, batch_named.names)

In [ ]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

In [ ]:
grey_named = (img_named * weights_aligned).sum('channels')
grey_named.shape, grey_named.names

In [ ]:
grey_plain = grey_named.rename(None)
grey_plain.shape, grey_plain.names

### dtypes

In [ ]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2,], [3, 4]], dtype=torch.short)

In [ ]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [ ]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

In [ ]:
points64 = torch.rand(4, dtype=torch.double)
points_short = points64.to(torch.short)
points_short

In [ ]:
points64 * points_short

### Tensor API

In [ ]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
a.shape, a_t.shape

In [ ]:
a_t = a.transpose(0, 1)
a_t.shape

## Storage
### Indexing into Storage

In [ ]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1]])
points.storage()

In [ ]:
points_storage = points.storage()
points_storage[0]

In [ ]:
points.storage()[1]

In [21]:
points_storage[0] = 2.2
points

NameError: name 'points_storage' is not defined

### Modifying Stored Values: In-place Ops

In [22]:
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## Tensor Metadata: Size, Offset, Stride
### Veiws of another Tensor's Storage

In [23]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [24]:
second_point = points[1]
print(second_point)
second_point.storage_offset()

tensor([5., 3.])


2

In [25]:
second_point.size()

torch.Size([2])

In [26]:
points.stride()

(2, 1)

In [27]:
second_point.stride()

(1,)

In [28]:
second_point[0] = 99.9
points

tensor([[ 4.0000,  1.0000],
        [99.9000,  3.0000],
        [ 2.0000,  1.0000]])

In [29]:
second_point = points[1].clone()
second_point[0] = -5.5
second_point

tensor([-5.5000,  3.0000])

In [30]:
points

tensor([[ 4.0000,  1.0000],
        [99.9000,  3.0000],
        [ 2.0000,  1.0000]])

### Transpose w/o Copying

In [31]:
points_t = points.t()
points_t

tensor([[ 4.0000, 99.9000,  2.0000],
        [ 1.0000,  3.0000,  1.0000]])

In [32]:
id(points.storage()) == id(points_t.storage())

True

### Higher Dimension Transposes

In [33]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape, transpose_t.shape

(torch.Size([3, 4, 5]), torch.Size([5, 4, 3]))

In [34]:
some_t.stride(), transpose_t.stride()

((20, 5, 1), (1, 5, 20))

### Contiguous Tensors

In [35]:
points.is_contiguous(), points_t.is_contiguous()

(True, False)

## Moving Tensors to the GPU
### Managing a Tensor's Device Attribute

In [38]:
#points_gpu = torch.tensor([[4., 1.], [5., 3.], [2., 1.]], device='cuda')

In [39]:
points

tensor([[ 4.0000,  1.0000],
        [99.9000,  3.0000],
        [ 2.0000,  1.0000]])

In [42]:
#points_gpu = points.to(device='cuda')

## Numpy Interoperability